---
title: "2023_Kronensicherung_Plesse_003_PTQ"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_PTQ
## ## PTQ: Daten der Elastometer (PicusTreeQinetic Data = PTQ)

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere das Paket treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Arbeitsumgebung vorbereiten


Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend das extra geschriebenen Pakete PTQ. Fehler beim Import dieses Pakets sind ggf. Bugs. Es nutzte eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
import numpy as np
import pandas as pd
import json
from IPython.display import Markdown, display
from pandas.api.types import CategoricalDtype

### IMPORT: Importieren eigenes Packet TreeQinetic

Das Packet TreeQinetic wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der TXT-Messdaten der Picus TreeQinetic Elastometer und Inclinometer der Firma IML Instrumenta Mechanik Labor Electronic GmbH geschrieben (https://www.iml-electronic.de/produkt/picus-treeqinetic/). Nachfolgend wird das Packet und einige dort definierten Klassen importiert.

In [2]:
import treeqinetic as ptq
# ptq.help() # Test

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [3]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

### IMPORT: Projekt Konfiguration laden

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [4]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    analyse_name,
    working_directory,
    data_path,
    data_export_directory,
    latex_export_directory
)

## IMPORT: TreeQinetic Daten laden

Aus dem Zugversuchsset wurden 4 Elastometer und 3 Inclinometer verwendet.

Die Elastometer waren auf dem rechten und linken Stämmling auf der Außenseite in zwei Ebenen platziert. Beim Zusammenziehen der Stämmlinge messen diese entsprechnd eine Faserdehnung, beim Ausschwingen der Stämmlinge über ihre Ruhelage hinaus nach außen eine Faserstauchung.

Die Inclinometer Daten werden hier ebefalls geladen, erschienen aber weniger geeignet zur Auswertung und werden entspricht nicht weiter berücksichtigt.

Die Funktion 'ptq.setup' erstellt div. Instanzen, die für das Paket notwendig sind (CONFIG, LOG_MANAGER, PLOT_MANAGER). 

Über die Klasse 'ptq.Series' wird eine neue Messreihe initialisiert und als 'ptq_series' gespeichert. Im Verzeichnis ptq_data_path finden sich die PTQ Daten als TXT von insgesamt 29 Messungen. Eine Datei enthält jeweils die Daten für alle Inclinometer und Elastometer.

In [5]:
ptq.setup(working_directory=working_directory, log_level="info", safe_logs_to_file=True)

ptq_data_path = data_path / 'PTQ/data_txt'
ptq_series = ptq.classes.Series(name=analyse_name, path=ptq_data_path)

# Relevante Elastometer
elasto_names = ["Elasto(90)", "Elasto(92)", "Elasto(95)", "Elasto(98)"]

2025-03-29 13:05:20 [INFO] kj_logger.update_config: <kj_logger.LogManager object at 0x0000027BFDDA8C50> initialized - update_config! Code: 000
2025-03-29 13:05:20 [INFO] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ptq was successfully created.
2025-03-29 13:05:20 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ptq!
2025-03-29 13:05:20 [INFO] kj_core.core_config.__init__: <CoreConfig>
  Package:            treeqinetic
  Working Directory:  C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ptq
  Plot Directory:     C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Ana

## ANALYSE: Explorative Datenanalyse

Übersicht über alle vom PTQ erfassten Daten über alle Messungen gemeinsam (Elastos und Inclinos).

In [6]:
ptq_df = ptq_series.get_measurements_df()
ptq_df

,ID,Nr,Datetime,Sec_Since_Start,Elasto(98),Elasto(95),Elasto(92),Elasto(90),Inclino(85)X,Inclino(85)Y,Inclino(83)X,Inclino(83)Y,Inclino(82)X,Inclino(82)Y,Inclino(81)X,Inclino(81)Y
0,1,0,2023-03-22 10:10:39.830198,0.000,NaN,-10.8,2.2,14.2,0.187,0.0,-0.180,-0.036,-0.055,0.061,-0.004,-0.004
1,1,1,2023-03-22 10:10:40.089198,0.259,-3.7,-11.1,3.1,14.7,0.174,0.0,-0.195,-0.017,-0.066,0.051,-0.002,-0.004
2,1,2,2023-03-22 10:10:40.329198,0.499,-4.3,-11.4,2.6,15.2,0.161,0.0,-0.205,0.016,-0.074,0.057,0.000,-0.003
3,1,3,2023-03-22 10:10:40.575198,0.745,-5.8,-11.5,3.3,15.5,0.123,0.0,-0.207,0.054,-0.082,0.077,-0.001,-0.006
4,1,4,2023-03-22 10:10:40.817198,0.987,-5.9,-11.8,3.4,15.5,0.098,0.0,-0.204,0.096,-0.087,0.101,-0.001,-0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,29,559,2023-03-22 16:12:54.492613,198.481,-0.7,-0.9,-7.3,-15.9,0.149,0.0,0.527,0.112,0.030,0.122,0.008,-0.002
17832,29,560,2023-03-22 16:12:54.734613,198.723,-0.4,-0.5,-8.0,-16.3,0.039,0.0,0.546,0.018,0.051,0.062,-0.001,-0.002
17833,29,561,2023-03-22 16:12:54.974613,198.963,-0.1,0.0,-8.2,-16.4,0.071,0.0,0.563,-0.070,0.071,0.006,0.000,-0.002
17834,29,562,2023-03-22 16:12:55.214613,199.203,0.2,0.7,-9.0,-16.3,0.151,0.0,0.579,-0.122,0.095,-0.042,-0.002,0.001


In [7]:
ptq_df.describe()

,ID,Nr,Datetime,Sec_Since_Start,Elasto(98),Elasto(95),Elasto(92),Elasto(90),Inclino(85)X,Inclino(85)Y,Inclino(83)X,Inclino(83)Y,Inclino(82)X,Inclino(82)Y,Inclino(81)X,Inclino(81)Y
count,17836.000000,17836.000000,17836,17836.000000,17461.000000,17468.000000,17657.000000,17676.000000,17448.000000,17448.0,17704.000000,17704.000000,17435.000000,17435.000000,17574.000000,17574.000000
mean,15.249047,338.457502,2023-03-22 13:34:17.235428864,85.133799,81.627536,105.786833,118.740437,191.147720,0.586046,0.0,0.735073,0.022071,-0.495272,-0.195005,0.009151,0.010782
min,1.000000,0.000000,2023-03-22 10:10:39.830198,0.000000,-192.900000,-119.100000,-111.000000,-217.800000,0.000000,0.0,-7.149000,-3.127000,-3.483000,-2.228000,-1.346000,-0.660000
25%,8.000000,153.000000,2023-03-22 11:11:37.444994048,38.984250,16.900000,8.400000,5.700000,13.475000,0.283000,0.0,0.424000,-0.102000,-0.922000,-0.397500,-0.006000,-0.005750
50%,16.000000,309.000000,2023-03-22 14:08:40.921139968,77.763500,95.100000,128.500000,139.500000,235.850000,0.557000,0.0,0.742000,0.000000,-0.522000,-0.221000,-0.002000,-0.001000
75%,23.000000,469.000000,2023-03-22 15:31:05.232380160,117.784500,137.500000,184.700000,213.100000,337.600000,0.848250,0.0,1.036000,0.127000,-0.070000,0.000000,0.001000,0.002000
max,29.000000,1392.000000,2023-03-22 16:12:55.455613,345.709000,234.600000,266.300000,291.500000,429.600000,7.029000,0.0,7.341000,4.525000,1.570000,1.147000,2.955000,0.526000
std,8.422199,242.404703,NaN,60.383660,64.555211,89.518611,101.978415,157.750503,0.392076,0.0,0.561636,0.253933,0.491294,0.271169,0.226831,0.075625


In [8]:
ptq_df_elasto_summary = ptq_df[elasto_names].describe()
ptq_df_elasto_summary

,Elasto(90),Elasto(92),Elasto(95),Elasto(98)
count,17676.000000,17657.000000,17468.000000,17461.000000
mean,191.147720,118.740437,105.786833,81.627536
std,157.750503,101.978415,89.518611,64.555211
min,-217.800000,-111.000000,-119.100000,-192.900000
25%,13.475000,5.700000,8.400000,16.900000
50%,235.850000,139.500000,128.500000,95.100000
75%,337.600000,213.100000,184.700000,137.500000
max,429.600000,291.500000,266.300000,234.600000


### PLOTTING: Daten aller Messungen bzw. Elastometer

Plotten der 4 verwendeten Elastometer in einem Plot für jede Messung. Die Plots werden im Verzeichnis ptq/plots/multi_sensors_vs_time_1/ abgelegt.
Für alle Messungen und Elastometer ist gut zu erkennen, wie die Faserdehnung während des zusammen ziehen der Stämmlinge zunimmt, dann im Moment des Realises plötzlich abfällt, um in Folge harmonisch gedämpft auszuschwingen (nährungsweise).

In [13]:
ptq_series.plot_measurement_sensors(sensor_names=elasto_names)

2025-03-29 13:05:22 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' successful.
2025-03-29 13:05:22 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_101814.txt', ID: 2, Start '2023-03-22 10:18:33.334820' to '2023-03-22 10:21:25.691820'' successful.
2025-03-29 13:05:22 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_102314.txt', ID: 3, Start '2023-03-22 10:26:25.086505' to '2023-03-22 10:28:50.847505'' successful.
2025-03-29 13:05:23 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_104400.txt', ID: 4, Start '2023-03-22 10:50:02.131987' to '2023-03-22 10:52:43.662987'' successful.
2025-03-29 13:05:23 [INFO] treeqinetic.class

## ANALYSE: Bestimmung von Schwingungsparametern

### Selektieren der relevanten Bereiche

Selektiere die Bereiche nach dem Release, bei dem es zu einer harmonisch gedämpften Schwingung kommt. Die Methode Series.get_oscillations sucht im Standardfall nach einem Bereich in den Messdaten mit einer Länge von 20 Sekunden. Der Anfangszeitpunkt wird durch einen plötzlichen Abfall der Dehnung auf unter Null bestimmt, bei dem die Steigung mindestens -25 beträgt. Die Suche nach dem Startzeitpunkt beginnt erst 60 Sekunden nach Messungsbeginn. Der entsprechende Code befindet sich im Paket classes/measurement.py und utils/select_oscillation.py. Die so isolierten Bereiche werden als Instanzen der Klasse Oscillation initialisiert. Weitere Parameter wie Amplitude, Frequenz und Dämpfung werden direkt berechnet.
Parameter:
- sensor_names: Eine Liste der Sensornamen, für die die Schwingungsdaten identifiziert werden sollen.
- min_time_default: Die Mindestzeitspanne nach Beginn der Messung, nach der die Suche nach Schwingungen beginnt (Standard: 60 Sekunden).
- min_value: Der minimale Wertschwellenwert, damit Sensordaten als gültig betrachtet werden.
- threshold_slope: Der Steigungsschwellenwert, um den Beginn einer Schwingung zu bestimmen.
- duration: Die Dauer, für die die Schwingungsdaten extrahiert werden sollen.

In [14]:
ptq_series.get_oscillations(
    sensor_names=elasto_names,
    min_time_default=60,
    min_value=50,
    threshold_slope=-50,
    duration=17.5
)

2025-03-29 13:05:28 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'
2025-03-29 13:05:28 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_101814.txt', ID: 2, Start '2023-03-22 10:18:33.334820' to '2023-03-22 10:21:25.691820'
2025-03-29 13:05:28 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_102314.txt', ID: 3, Start '2023-03-22 10:26:25.086505' to '2023-03-22 10:28:50.847505'
2025-03-29 13:05:28 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_104400.txt', ID: 4, Start '2023-03-22 10:50:02.131987' to '2023-03-22 10:52:43.662987'
2025-03-29 13:05:28 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_105549.txt', ID: 5, Start '2023-03-22 10:55:59.

### Selektion optisch prüfen in Plots

Plotten der relevanten Sensoren bzw. der selektierten Bereiche. Die Plots werden im Verzeichnis ptq/plots/select_oscillations_single/ bzw. ptq/plots/select_oscillations_combined/ gespeichert. In einem Combined-Plot werden alle 4 Elastometer einer Messung gemeinsam dargestellt. Hier wird manuell anhand der Plots geprüft, ob für alle Messungen und Sensoren der richtige Bereich ausgewählt wurde.

In [15]:
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=False)
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=True)

2025-03-29 13:05:29 [INFO] treeqinetic.classes.series.plot_oscillations_for_measurements: Plot Oscillations for Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'
2025-03-29 13:05:30 [INFO] treeqinetic.classes.measurement._plot_single_oscillations: plot_select_oscillation for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' for Elasto(90) successful.
2025-03-29 13:05:31 [INFO] treeqinetic.classes.measurement._plot_single_oscillations: plot_select_oscillation for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' for Elasto(92) successful.
2025-03-29 13:05:32 [INFO] treeqinetic.classes.measurement._plot_single_oscillations: plot_select_oscillation for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' for Ela

### Anpassen der harmonisch gedämpften Schwingung

Aus der PTQ-Messreihe 'ptq_series' wird über `get_oscillations_list` für alle Messungen, getrennt für jeden Sensor (Elastometer), die `Oscillation`-Instanz in eine Liste zusammengeführt. Entsprechend gibt es für jede PTQ-Messung 4 `Oscillation`-Instanzen (für die 4 Elastometer).

`oscillation.fit` passt alle Schwingungsdaten mit einer allgemeinen Funktion für harmonisch gedämpfte Schwingungen an:
\[
y(t) = A \cdot e^{-\delta t} \cdot \cos(\omega_d \cdot t + \phi) + y_0
\]

#### Parameterbeschreibung:
- `A` (Anfangsamplitude): Der Anfangswert der Amplitude der Schwingung. Dieser Parameter bestimmt die initiale Höhe der Schwingungsamplitude.
- `δ` (Dämpfungskoeffizient): Dieser Wert bestimmt, wie schnell die Amplitude der Schwingung mit der Zeit abnimmt. Ein höherer Wert führt zu einer schnelleren Dämpfung der Schwingung.
- `ω_d` (gedämpfte Kreisfrequenz): Die Frequenz der gedämpften Schwingung in Radiant pro Sekunde. Dieser Parameter bestimmt, wie schnell die Schwingung oszilliert.
- `φ` (Phasenwinkel): Der Anfangsphasenwinkel der Schwingung. Dieser Wert bestimmt den Startpunkt der Schwingung im Schwingungszyklus.
- `y_0` (Vertikale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve vertikal und ermöglicht es, die Schwingung an die mittlere Position der Daten anzupassen.
- `t_0` (Horizontale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve horizontal über die Zeit und ermöglicht es, die Schwingung an den spezifischen Startpunkt der gemessenen Schwingung anzupassen.

(siehe `ptq/analyse/fitting_function.py`)

#### Zusätzliche Parameter und Konfigurationen:
- **Startwerte und Grenzwerte:** Für die Optimierung der Parameter in `scipy.curve_fit` werden Startwerte und Grenzwerte für jeden Parameter übergeben (in `ptq/config.py` definiert).
- **Qualitätsmetriken:** Zur Bewertung der Anpassungsgüte werden Metriken wie MAE (mittlerer absoluter Fehler), RMSE (Root Mean Square Error), und \( R^2 \) (Bestimmtheitsmaß) berechnet. Zusätzlich werden normalisierte Varianten (NRMSE und NMAE) zur besseren Vergleichbarkeit verwendet.
- **Warnungen bei Überschreitung der Grenzwerte:** Wenn die für eine Metrik definierten Grenzwerte überschritten werden, wird eine Warnung im Log-Protokoll vermerkt, um auf mögliche Probleme bei der Anpassung hinzuweisen (in `ptq/config.py` definiert). Auf Basis dieser Warnung können:
  - Start- und Grenzwerte sowie die Methodik angepasst werden.
  - Betroffene Datensätze später ausgeschlossen werden, um fehlerhafte Anpassungen zu vermeiden.
- **Interpolation:** Diese Option aktiviert die Interpolation der Datenpunkte, um eine ausreichende Dichte für `curve_fit` zu gewährleisten. Hierbei wird `scipy.interpolate.PchipInterpolator` verwendet, um Über- und Unterschwingungen, die nicht in den Originaldaten vorhanden sind, zu vermeiden. Nach optischer Prüfung zeigte diese Methode die besten Ergebnisse.

#### Visualisierungsoptionen:
- **Plot:** Wenn auf `True` gesetzt, wird für jede Oscillation ein Plot der angepassten Funktion zusammen mit den Originaldaten erstellt und in `working_dir/PTQ/plots/` gespeichert.
- **Plot-Fehlerverteilung:** Wenn `plot_error` auf `True` gesetzt ist, wird ein Histogramm der Fehlerverteilung (Residuen) für jeden Fit erstellt und ebenfalls in `working_dir/PTQ/plots/` gespeichert.

In [25]:
ptq_oscillations_ls = ptq_series.get_oscillations_list()

initial_param = {
    "initial_amplitude": 170,
    "damping_coeff": 0.32,
    "frequency_damped": 0.44,
    "phase_angle": 0,
    "y_shift": 0,
    "x_shift": 0
}

param_bounds = {
    "initial_amplitude": (150, 250),
    "damping_coeff": (0.1, 1),
    "frequency_damped": (0.35, 0.58),
    "phase_angle": (-0.2, 0.2),
    "y_shift": (-60, 60),
    "x_shift": (-0.25, 0.75),
}

metrics_warning = {
    "pearson_r": (0.75, 1),
    "nrmse": (0, np.inf),
    "mae": (0, np.inf),
    "nmae": (0, 0.10)
}

for oscillation in ptq_oscillations_ls:
    oscillation.fit(
        initial_param,
        param_bounds,
        optimize_criterion="mae",
        metrics_warning=metrics_warning,
        plot=False,
        plot_error=False,
        dir_add="",
        interpolate=True
    )

2025-03-29 13:11:37 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(90)'' successful.
2025-03-29 13:11:38 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(92)'' successful.
2025-03-29 13:11:38 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(95)'' successful.
2025-03-29 13:11:38 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(98)'' successful.
2025-03-29 13:11:39 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_101814.txt', ID: 2, Sensor: Elasto(90)'' successful.
2025-03-29 13:11:39 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_101814.txt', ID: 2, Sensor: Elasto(92)'' successful.
2025-03-29 13:11:39 [INFO] t

### Fehlerverteilung der Funktionsanpassung an Messdaten

Die Funktion sammelt für alle Oscillation-Objekte die Fehler-Arrays der Anpassung und normalisiert die Fehler (um Unterschiede in der Skalierung zu entfernen).
Anschließend werden die Fehler für alle Messungen A) für alle Sensoren gemeinsam und B) getrennt für jeden Sensor geplotet. Es werden Q-Q-Plot, Violin-Plot und Histogramme für den gleichen Sachverhalt erstellt und in `working_directory\PTQ\plots\series_osc_errors` abgelegt.

- `trim_hist_percent`: Beschneidet die Daten Links und Rechts um die äußersten x Prozent, da die Verteilung im Zentrum sonst kaum zu bewerten ist. Wirkt sich nur auf die Histogramme aus.

In [17]:
all_normalized_errors = ptq_series.plot_osc_errors(plot_qq=True, plot_violin=True, plot_hist=True, hist_trim_percent=5)

## ANALYSE: Metadaten bzw. Zusammenfassung aller Dehnungs- und Schwingungsdaten

Bewerte die Güte der Anpassung

In [18]:
osc_optimization_details_df = ptq_series.get_osc_optimization_details_df()
osc_optimization_details_df.describe()

,id,nit,nfev,status,fun,pearson_r,p_value,r2,mse,rmse,nrmse,cv,mae,nmae
count,116.000000,116.000000,116.000000,116.0,116.000000,116.000000,1.160000e+02,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,15.000000,42.474138,606.586207,0.0,9.073974,0.889016,4.420063e-82,0.759775,350.973103,16.916159,0.053132,-0.491524,9.073974,0.029283
std,8.402898,13.845396,177.911168,0.0,3.310947,0.084336,4.760553e-81,0.174199,541.187781,8.085809,0.017778,20.647209,3.310947,0.009197
min,1.000000,16.000000,252.000000,0.0,4.860754,0.577722,0.000000e+00,0.136988,50.590635,7.112709,0.025014,-164.381615,4.860754,0.014277
25%,8.000000,33.750000,460.250000,0.0,7.095709,0.853768,0.000000e+00,0.685030,145.843592,12.076532,0.040790,-1.476578,7.095709,0.021926
50%,15.000000,41.500000,605.500000,0.0,8.300605,0.919387,0.000000e+00,0.810573,216.465115,14.712081,0.049523,-0.509270,8.300605,0.028352
75%,22.000000,50.000000,743.750000,0.0,10.230048,0.943763,5.853015e-245,0.876082,393.131995,19.827412,0.064283,2.285988,10.230048,0.034944
max,29.000000,82.000000,1078.000000,0.0,30.641411,0.987644,5.127273e-80,0.973909,5501.754846,74.173815,0.152001,95.362596,30.641411,0.062792


'ptq_series.get_oscillations_df' fasst aus allen Oscillation-Instanzen der Messreihe ('ptq_series') die Schwingungsparameter als pandas.DataFrame zusammen

In [19]:
ptq_metadata_df = ptq_series.get_oscillations_df()

In [20]:
metrics_warning_count = ptq_metadata_df['metrics_warning'].sum()
print(f"metrics_warning_count: {metrics_warning_count}")

metrics_warning_count: 9


## IMPORT: Datendokumentation laden

In [21]:
ptq_data_dict = ptq_series.create_oscillations_data_dict()

2025-03-29 13:11:13 [INFO] treeqinetic.classes.series.create_oscillations_data_dict: Data dictionary loaded with 28 entries.


## ANALYSE: Datendokumentation für Metadaten ansehen

In [22]:
# In DataFrame umwandeln
ptq_data_dict_df= build_data_dict_df(ptq_data_dict)

# In Markdown umwandeln und anzeigen
md_text = ptq_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable            | Kategorie      | Zeichen                            | Deutsch                                    | Datentyp   | Einheit   | Beschreibung                                                            |
|----|---------------------|----------------|------------------------------------|--------------------------------------------|------------|-----------|-------------------------------------------------------------------------|
|  0 | id                  | ptq            | ID                                 | ID Messung                                 | int64      | -         | Eindeutige ID der Messung                                               |
|  1 | file_name           | ptq            | filename                           | Dateiname                                  | object     | -         | Name der Quelldatei der Messung                                         |
|  2 | sensor_name         | ptq            | sensor                             | Sensorname                                 | object     | -         | Bezeichnung des Elastometers                                            |
|  3 | sample_rate         | ptq            | $f_s$                              | Abtastrate                                 | float64    | Hz        | Abtastrate der Datenaufzeichnung                                        |
|  4 | max_strain          | ptq            | $\Delta L_{\mathrm{max}}$          | Dehnung max. gemessen                      | float64    | $\mu$m    | Gemessene maximale absolute Randfaserdehnung                            |
|  5 | max_compression     | ptq            | $\Delta L_{\mathrm{comp,max}}$     | Stauchung max. gemessen                    | float64    | $\mu$m    | Gemessene maximale absolute Randfaserstauchung                          |
|  6 | max_strain_osc      | ptq            | $\Delta L_{\mathrm{max,osc}}$      | Maximale Dehnung (Oszillation)             | float64    | $\mu$m    | Gemessene maximale absolute Randfaserdehnung innerhalb der Schwingung   |
|  7 | max_compression_osc | ptq            | $\Delta L_{\mathrm{comp,max,osc}}$ | Maximale Stauchung (Oszillation)           | float64    | $\mu$m    | Gemessene maximale absolute Randfaserstauchung innerhalb der Schwingung |
|  8 | m_amplitude         | ptq_osc        | $mA$                               | Manuelle Amplitude                         | float64    | $\mu$m    | Manuell berechnete Amplitude über den Schwingungsabschnitt              |
|  9 | m_amplitude_2       | ptq_osc        | $mA_2$                             | Manuelle Amplitude 2                       | float64    | $\mu$m    | Manuell berechnete Amplitude zwischen 2. Peak und Minimum               |
| 10 | metrics_warning     | ptq_osc        | warning                            | Fit-Warnung                                | bool       | -         | Warnung, wenn Qualitätsmetrik Schwellenwerte unterschreitet             |
| 11 | initial_amplitude   | ptq_osc        | $A$                                | Anfangsamplitude                           | float64    | $\mu$m    | Initiale Amplitude der angepassten Schwingung                           |
| 12 | damping_coeff       | ptq_osc        | $\delta$                           | Dämpfungskoeffizient                       | float64    | 1/s       | Koeffizient der exponentiellen Dämpfung                                 |
| 13 | damping_ratio       | ptq_osc        | $D$                                | Dämpfungsgrad                              | float64    | -         | Verhältnis von Dämpfung zu Frequenz                                     |
| 14 | frequency_damped    | ptq_osc        | $f_{\mathrm{d}}$                   | Gedämpfte Frequenz                         | float64    | Hz        | Frequenz der gedämpften Schwingung                                      |
| 15 | frequency_undamped  | ptq_osc        | $f_0$                              | Ungedämpfte Frequenz                       | float64    | Hz        | Frequenz der ungedämpften Schwingung                                    |
| 16 | pearson_r           | ptq_osc_metric | $r$                                | Pearson-Korrelation                        | float64    | -         | Korrelationskoeffizient der Anpassung                                   |
| 17 | nmae                | ptq_osc_metric | $\mathrm{NMAE}$                    | Normalisierter MAE                         | float64    | -         | Normalisierter mittlerer absoluter Fehler                               |
| 18 | nrmse               | ptq_osc_metric | $\mathrm{NRMSE}$                   | Normalisierter RMSE                        | float64    | -         | Normalisierter mittlerer quadratischer Fehler                           |
| 19 | phase_angle         | ptq_osc        | $\varphi$                          | Phasenwinkel                               | float64    | rad       | Anfangsphase der Schwingung                                             |
| 20 | x_shift             | ptq_osc        | $t_0$                              | Zeitverschiebung                           | float64    | s         | Horizontale Verschiebung der Schwingung                                 |
| 21 | y_shift             | ptq_osc        | $y_0$                              | Vertikaler Versatz                         | float64    | $\mu$m    | Vertikaler Offset der Schwingung                                        |
| 22 | p_value             | ptq_osc_metric | p                                  | p-Wert                                     | float64    | -         | Signifikanzwert der Pearson-Korrelation                                 |
| 23 | r2                  | ptq_osc_metric | $R^2$                              | Bestimmtheitsmaß                           | float64    | -         | Erklärte Varianz durch das Modell                                       |
| 24 | mse                 | ptq_osc_metric | $\mathrm{MSE}$                     | Mittlerer quadratischer Fehler             | float64    | -         | Mittlerer quadratischer Fehler der Anpassung                            |
| 25 | rmse                | ptq_osc_metric | $\mathrm{RMSE}$                    | Wurzel des mittleren quadratischen Fehlers | float64    | -         | Wurzel aus dem mittleren quadratischen Fehler                           |
| 26 | cv                  | ptq_osc_metric | $\mathrm{CV}$                      | Variationskoeffizient                      | float64    | -         | Verhältnis von RMSE zum Mittelwert                                      |
| 27 | mae                 | ptq_osc_metric | $\mathrm{MAE}$                     | Mittlerer absoluter Fehler                 | float64    | -         | Durchschnittlicher absoluter Fehler der Anpassung                       |

## EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [23]:
# DataFrame als Feather
ptq_metadata_df.to_feather(data_export_directory / "ptq.feather")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "ptq_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(ptq_data_dict, f,  indent=4, ensure_ascii=False)
    
ptq_metadata_df.to_csv(data_export_directory / "ptq.csv", sep=";", index=False, encoding="utf-8")

## LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [24]:
ptq_data_dict_df = build_data_dict_df(ptq_data_dict, escape_index=True, select_latex_fields=True)

latex_string = ptq_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - PTQ Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ptq_daten_dokumentation.tex
